## Import stuff

In [2]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras import layers
import shutil
import random
from PIL import Image
import numpy as np
import logging
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import torch 
import torch.jit
import torch.nn as nn 
import torchvision
from torchvision import transforms, datasets
import sklearn.metrics as metrics
from sklearn.svm import SVC
from sklearn import svm
from sklearn import datasets
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import ClusterCentroids
logger = tf.get_logger()
logger.setLevel(logging.ERROR)
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

print(tf.__version__)

from tensorflow.python.client import device_lib
print("tensorflow doesn't detects GPU: ")
print(device_lib.list_local_devices() )
print(tf.test.gpu_device_name())
print(tf.config.list_physical_devices())

print("torch detects GPU: " , torch.cuda.is_available())

oversampler = RandomOverSampler(random_state=0)
undersampler = ClusterCentroids(random_state=0)


def get_score(df_true, df_submission):
    for i in range(df_true.shape[1]):
        print("score for col ",i,": " , metrics.roc_auc_score(df_true[:,i], df_submission[:,i]))


2.3.0
tensorflow doesn't detects GPU: 
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1366717752328370373
]

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
torch detects GPU:  True


## Load training data from csv


In [3]:
features_df_orig = pd.read_csv("train_features_fixed.csv")
features_df_normalized = pd.read_csv("train_features_normalized.csv")
labels_df = pd.read_csv("train_labels.csv")


# We can use a different version of the training data for each task
X_orig = np.array(features_df_orig.iloc[:, 2:].values,  dtype=float).reshape(227940//12,35*12)
X_normalized = np.array(features_df_normalized.iloc[:, 2:].values,  dtype=float).reshape(227940//12,35*12)
X_small = np.array(features_df_normalized.iloc[:, 2:].values,  dtype=float)[::12]



#use train to validation ratio of 0.8
X_orig_train = X_orig[:int(X_orig.shape[0]*0.8)]
X_orig_valid = X_orig[int(X_orig.shape[0]*0.8):]
X_normalized_train = X_normalized[:int(X_normalized.shape[0]*0.8)]
X_normalized_valid = X_normalized[int(X_normalized.shape[0]*0.8):]
X_small_train = X_small[:int(X_small.shape[0]*0.8)]
X_small_valid = X_small[int(X_small.shape[0]*0.8):]

Y1 = np.array(labels_df.iloc[:, 1:11].values,  dtype=float)
Y1_train = Y1[:int(Y1.shape[0]*0.8)]
Y1_valid = Y1[int(Y1.shape[0]*0.8):]

Y2 = np.array(labels_df.iloc[:, 11].values,  dtype=float)
Y2_train = Y2[:int(Y2.shape[0]*0.8)]
Y2_valid = Y2[int(Y2.shape[0]*0.8):]

Y3 = np.array(labels_df.iloc[:, 12:].values,  dtype=float)
Y3_train = Y3[:int(Y3.shape[0]*0.8)]
Y3_valid = Y3[int(Y3.shape[0]*0.8):]

print("X_orig shape: ", X_orig.shape)
print("X_normalized shape: ", X_normalized.shape)
print("X_small shape: ", X_small.shape)
print("Y1 shape: ", Y1.shape)
print("Y2 shape: ", Y2.shape)
print("Y3 shape: ", Y3.shape)


X_orig shape:  (18995, 420)
X_normalized shape:  (18995, 420)
X_small shape:  (18995, 35)
Y1 shape:  (18995, 10)
Y2 shape:  (18995,)
Y3 shape:  (18995, 4)


## TASK 1: ORDERING OF MEDICAL TEST

In [ ]:
# X_normalized_undersampling, Y1_undersampling = undersampler.fit_resample(X_small_train, Y1_train)

# print(Y1_undersampling[Y1_undersampling==1].shape)
# print(Y1_undersampling[Y1_undersampling==0].shape)

In [5]:
inputs_dim = X_normalized.shape[1]
outputs_dim = Y1.shape[1]

model1 = tf.keras.Sequential([
  tf.keras.layers.Dense(units=100, input_shape=[inputs_dim], activation='sigmoid'),
  tf.keras.layers.Dense(units=25, activation='sigmoid'),
  tf.keras.layers.Dense(units=outputs_dim)
])

model1.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(0.1), metrics=['mean_squared_error'])

history = model1.fit(X_normalized_train, Y1_train, epochs=10) # , verbose=False
print("Finished training the model")

Epoch 1/10
475/475 [==============================] - 0s 675us/step - loss: 0.1484 - mean_squared_error: 0.1484
Epoch 2/10
475/475 [==============================] - 0s 681us/step - loss: 0.1430 - mean_squared_error: 0.1430
Epoch 3/10
475/475 [==============================] - 0s 622us/step - loss: 0.1449 - mean_squared_error: 0.1449
Epoch 4/10
475/475 [==============================] - 0s 661us/step - loss: 0.1443 - mean_squared_error: 0.1443
Epoch 5/10
475/475 [==============================] - 0s 636us/step - loss: 0.1431 - mean_squared_error: 0.1431
Epoch 6/10
475/475 [==============================] - 0s 632us/step - loss: 0.1436 - mean_squared_error: 0.1436
Epoch 7/10
475/475 [==============================] - 0s 649us/step - loss: 0.1451 - mean_squared_error: 0.1451
Epoch 8/10
475/475 [==============================] - 0s 615us/step - loss: 0.1436 - mean_squared_error: 0.1436
Epoch 9/10
475/475 [==============================] - 0s 626us/step - loss: 0.1436 - mean_squared_error:

## TASK 1: EVALUATION

In [6]:
pred1 = model1.predict(X_normalized_valid)
print(np.var(pred1))
for i in range(Y1_valid.shape[1]):
    print("score for col ",i,": " , metrics.roc_auc_score(Y1_valid[:,i], pred1[:,i]))



0.026957536
score for col  0 :  0.5069146988517818
score for col  1 :  0.4890370438253503
score for col  2 :  0.5069922578500399
score for col  3 :  0.5
score for col  4 :  0.5
score for col  5 :  0.5
score for col  6 :  0.5040007788161993
score for col  7 :  0.49672649953023784
score for col  8 :  0.5103490053144917
score for col  9 :  0.5


## TASK 2: SEPSIS PREDICTION

In [7]:
X_small_undersampling, Y2_undersampling = undersampler.fit_resample(X_small_train, Y2_train)

print(Y2_undersampling[Y2_undersampling==1].shape)
print(Y2_undersampling[Y2_undersampling==0].shape)

(886,)
(886,)


In [15]:
#gaussian kernel (pretty bad)
# kernel = "rbf"
# reg = np.power(10., -1.5)
# bandwidth = np.power(10., -0.6)
# gamma = np.power(10., -bandwidth)
# coef0 = 0
# tol = 1e-1

# model2 = svm.SVC(kernel=kernel, C=np.power(10., -reg), gamma=gamma, coef0=coef0, tol=tol, random_state=10)
            
# model2.fit(X_small_undersampling[:10000], Y2_undersampling[:10000])


#polynomial kernel
degree = 3
l2_coef = 0.01

model2 = make_pipeline(
    PolynomialFeatures(degree),
    Ridge(alpha=l2_coef, fit_intercept=False, solver="svd"))
    
model2.fit(X_small_undersampling[:1000], Y2_undersampling[:1000])



Pipeline(steps=[('polynomialfeatures', PolynomialFeatures(degree=3)),
                ('ridge',
                 Ridge(alpha=0.01, fit_intercept=False, solver='svd'))])

## TASK 2: EVALUATION

In [16]:
pred2 = model2.predict(X_small_valid)
print(np.var(pred2))
print(pred2.shape)
print(Y2_valid.shape)
print(Y2_valid[Y2_valid==1].shape)
print(Y2_valid[Y2_valid==0].shape)

print("score : " , metrics.roc_auc_score(Y2_valid, pred2.reshape(-1,1)))

0.06696288840906799
(3799,)
(3799,)
(202,)
(3597,)
score :  0.5886960255658594


## TASK 3: KEYS VITALS SIGNS PREDICTION

In [11]:
inputs_dim = X_orig.shape[1]
outputs_dim = Y3.shape[1]

model3 = tf.keras.Sequential([
  tf.keras.layers.Dense(units=100, input_shape=[inputs_dim], activation='relu'),
  tf.keras.layers.Dense(units=25, activation='relu'),
  tf.keras.layers.Dense(units=outputs_dim)
])

model3.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(0.1), metrics=['mean_squared_error'])

history = model3.fit(X_normalized_train, Y3_train, epochs=20) # , verbose=False
print("Finished training the model")

Epoch 1/20
475/475 [==============================] - 0s 621us/step - loss: 114.8926 - mean_squared_error: 114.8926
Epoch 2/20
475/475 [==============================] - 0s 731us/step - loss: 56.8139 - mean_squared_error: 56.8139
Epoch 3/20
475/475 [==============================] - 0s 664us/step - loss: 50.0968 - mean_squared_error: 50.0968
Epoch 4/20
475/475 [==============================] - 0s 609us/step - loss: 48.8615 - mean_squared_error: 48.8615
Epoch 5/20
475/475 [==============================] - 0s 647us/step - loss: 49.7978 - mean_squared_error: 49.79780s - loss: 52.5729 - mean_squared_error: 5
Epoch 6/20
475/475 [==============================] - 0s 649us/step - loss: 55.3849 - mean_squared_error: 55.3849
Epoch 7/20
475/475 [==============================] - 0s 619us/step - loss: 48.2290 - mean_squared_error: 48.2290
Epoch 8/20
475/475 [==============================] - 0s 604us/step - loss: 48.8632 - mean_squared_error: 48.8632
Epoch 9/20
475/475 [========================

## TASK 3: EVALUATION

In [12]:
pred3 = model3.predict(X_normalized_valid)
for i in range(Y3_valid.shape[1]):
    print("score for col ",i,": " , 0.5 + 0.5 * np.maximum(0, metrics.r2_score(Y3_valid[i], pred3[i])))



score for col  0 :  0.985809041303258
score for col  1 :  0.9591870842407233
score for col  2 :  0.8747249899278796
score for col  3 :  0.9691530467926881


## Writing results

In [37]:
print(pd.read_csv("test_features.csv").shape)
print(pd.read_csv("test_features_normalized.csv").shape)
test_features_df_normalized = pd.read_csv("test_features_normalized.csv")

test_X_normalized = np.array(test_features_df_normalized.iloc[:, 2:].values,  dtype=float).reshape(151968//12,35*12)
test_X_small = np.array(test_features_df_normalized.iloc[:, 2:].values,  dtype=float)[::12]

print(test_X_normalized.shape)
print(test_X_small.shape)


pred1 = model1.predict(test_X_normalized)
pred2 = model2.predict(test_X_small)
pred3 = model3.predict(test_X_normalized)




(151968, 37)
(151968, 37)
(12664, 420)
(12664, 35)


In [38]:
sample = pd.read_csv("sample.csv")
print(sample.shape)
submit =  pd.DataFrame(np.zeros((12664,16)), columns=[sample.columns])
print(submit.shape)
print(pred1.shape)
print(pred2.shape)
print(pred3.shape)
# print(submit)


submit.iloc[:,0] = sample.iloc[:,0]
submit.iloc[:,1:11] = pred1
submit.iloc[:,11] = pred2
submit.iloc[:,12:] = pred3

submit.to_csv('submit.zip', index=False, float_format='%.3f', compression='zip')
submit.to_csv('submit.csv', index=False, float_format='%.3f')

(12664, 16)
(12664, 16)
(12664, 10)
(12664,)
(12664, 4)
